# Review of julia itterative solver


![ssor_image](algorithm_4.png)

In [ ]:
"""
struct SparseMatrixCSC{Tv,Ti<:Integer} <: AbstractSparseMatrix{Tv,Ti}
    m::Int                  # Number of rows
    n::Int                  # Number of columns
    colptr::Vector{Ti}      # Column i is in colptr[i]:(colptr[i+1]-1)
    rowval::Vector{Ti}      # Row indices of stored values
    nzval::Vector{Tv}       # Stored values, typically nonzeros
end
"""


In [3]:
  
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate

using SparseArrays

struct DiagonalIndices{Tv, Ti <: Integer}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::Vector{Ti}

    function DiagonalIndices{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti}
        # Check square?
        diag = Vector{Ti}(undef, A.n)

        for col = 1 : A.n
            r1 = Int(A.colptr[col])
            r2 = Int(A.colptr[col + 1] - 1)
            r1 = searchsortedfirst(A.rowval, col, r1, r2, Base.Order.Forward)
            if r1 > r2 || A.rowval[r1] != col || iszero(A.nzval[r1])
                throw(LinearAlgebra.SingularException(col))
            end
            diag[col] = r1
        end 

        new(A, diag) #
    end
end

DiagonalIndices(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti} = DiagonalIndices{Tv,Ti}(A)
#밖에서도 함수로 쓸수있게 따로 빼준건가?

#DiagonalIndices 특징은 Sparse matrix의 nz diagonal element vector로 변환시키며,
#diagonal term에 0이 있으면 안된다.
#원 matrix을 담고 있는 구조체임.
# A.nz는 col 순서대로 적어내린 values
# A.row는 nz 기준 해당 row index
# A.cor은 i에  해당하는 nz index 범위를 찍어줌

@inline getindex(d::DiagonalIndices, i::Int) = d.diag[i]
#get index를 선언해줘야 forwardsub!가 돌아갔음..




getindex (generic function with 200 methods)

In [56]:
"""
Forward substitution for the FastLowerTriangular type
"""
function forward_sub!(F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix
    @inbounds for col = 1 : A.n
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # ok
        for i = idx + 1 : (A.colptr[col + 1] - 1) #colptr인데 lower triangular이기 때문에 해당 col의 diagonal 아래 개수가나옴.
            x[A.rowval[i]] -= A.nzval[i] * x[col] # 이 term으로 x[n] 계산할때 그이전텀들이 다 마이너스 되어서 있음. 
        end
    end
    x
end

# solve α × A(x- β × y) = b 
function forward_sub!(α, F::FastLowerTriangular, x::AbstractVector, β, y::AbstractVector)
    A = F.matrix

    @inbounds for col = 1 : A.n

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] = α * x[col] / A.nzval[idx] + β * y[col]

        # Substitute next values involving x[col]
        for i = idx + 1 : (A.colptr[col + 1] - 1)
            x[A.rowval[i]] -= A.nzval[i] * x[col]
        end
    end

    x
end

#(α×A)×(β×y)  
function mul!(α::T, O::OffDiagonal, x::AbstractVector, β::T, y::AbstractVector) where {T}
    # Specialize for β = 0 and β = 1
    A = O.matrix

    if β != one(T)
        if iszero(β)
            fill!(y, zero(T))
        else
            lmul!(β, y) #그냥 베타 곱해주는 함수
        end
    end

    @inbounds for col = 1 : A.n
        αx = α * x[col]
        diag_index = O.diag[col]
        for j = A.colptr[col] : diag_index - 1 #off diagonal 이기때문에 해당 제외 데이터 개수
            y[A.rowval[j]] += A.nzval[j] * αx #y를 솔루션이라고 하면 해당하는 
        end
        #off diagonal 하나 빼는게 힘들어서 이지랄/..
        for j = diag_index + 1 : A.colptr[col + 1] - 1
            y[A.rowval[j]] += A.nzval[j] * αx
        end
    end

    y
end


forward_sub!

In [57]:
Matrix(LT.matrix)

4×4 Array{Float64,2}:
 1.0  0.0  0.0   0.0
 1.0  1.0  0.0   0.0
 0.0  1.0  4.0   0.0
 1.0  0.0  0.0  10.0

In [58]:
LT.matrix.colptr

5-element Array{Int64,1}:
 1
 4
 6
 7
 8

## Test code for forwardsub 

In [55]:

struct FastLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end
 
A = sparse([1.0 2 3 0; 1 1 0 0;0 1 4 1; 1 0 0 10])
Da = DiagonalIndices(A)

display("matrix A")
display(Matrix(A))
display("matrix A diag")
display(Da.diag)


A_temp = LowerTriangular(A)
#A_temp = UpperTriangular(A)
A_temp = sparse(A_temp)
D_a = DiagonalIndices(A_temp)
LT = FastLowerTriangular(D_a.matrix,D_a)
b = [1.1 ;2.0 ;3.0 ;4.0]
display(Matrix(A_temp))
display(b)


"matrix A"

4×4 Array{Float64,2}:
 1.0  2.0  3.0   0.0
 1.0  1.0  0.0   0.0
 0.0  1.0  4.0   1.0
 1.0  0.0  0.0  10.0

"matrix A diag"

4-element Array{Int64,1}:
  1
  5
  8
 10

4×4 Array{Float64,2}:
 1.0  0.0  0.0   0.0
 1.0  1.0  0.0   0.0
 0.0  1.0  4.0   0.0
 1.0  0.0  0.0  10.0

4-element Array{Float64,1}:
 1.1
 2.0
 3.0
 4.0

In [45]:
Matrix(LT.matrix)

4×4 Array{Float64,2}:
 1.0  0.0  0.0   0.0
 0.0  1.0  0.0   0.0
 0.0  1.0  4.0   0.0
 1.0  0.0  0.0  10.0

In [52]:
LT.diag

DiagonalIndices{Float64,Int64}(
  [1, 1]  =  1.0
  [4, 1]  =  1.0
  [2, 2]  =  1.0
  [3, 2]  =  1.0
  [3, 3]  =  4.0
  [4, 4]  =  10.0, [1, 3, 5, 6])

In [51]:
LT.diag[4]

6

In [47]:
LT.diag[1]

1

# miscellaneous

In [6]:
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate
using LinearAlgebra

A_ = sparse([1.0 0 3 0; 2 1 0 0;0 1 4 1; 0 0 1 10])
Da_ = DiagonalIndices(A_)

display("matrix A_")
display(Matrix(A_))
display("matrix A_ diag")
display(Da_.diag)

A = sparse([1.0 2 3 0; 0 1 0 0;0 1 4 1; 1 0 0 10])
Da = DiagonalIndices(A)

display("matrix A")
display(Matrix(A))
display("matrix A diag")
display(Da.diag)


#A.colptr #col_i에서 매칭된 element 개수와 위치 with A.nzval[col[i] : col[i+1]-1]
#A.diag는 col base counting에서 n-th diag elment까지 nz element의 개수(자기 포함)
#diag는 nzval map을 이용해서 diag로 접근가능
#get index 참고하면 알 수 있음.

"matrix A_"

4×4 Array{Float64,2}:
 1.0  0.0  3.0   0.0
 2.0  1.0  0.0   0.0
 0.0  1.0  4.0   1.0
 0.0  0.0  1.0  10.0

"matrix A_ diag"

4-element Array{Int64,1}:
 1
 3
 6
 9

"matrix A"

4×4 Array{Float64,2}:
 1.0  2.0  3.0   0.0
 0.0  1.0  0.0   0.0
 0.0  1.0  4.0   1.0
 1.0  0.0  0.0  10.0

"matrix A diag"

4-element Array{Int64,1}:
 1
 4
 7
 9

In [74]:
A.rowval

9-element Array{Int64,1}:
 1
 4
 1
 2
 3
 1
 3
 3
 4

The following two types are equivalent – one with a default constructor, the other with an explicit constructor:

~~~
julia> struct T1
           x::Int64
       end

julia> struct T2
           x::Int64
           T2(x) = new(x)
       end
~~~
